In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats)
import time

In [2]:
pbp2022=pd.read_csv(filepath_or_buffer="Data/2021-22_pbp.csv")

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [3]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season="2021-22",season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season="2021-22",
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [4]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [5]:
hustle_70percent_gp.nlargest(5,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
37,Blake Griffin,0.968421
347,Kevin Love,0.573451
195,Garrison Mathews,0.547529
425,Moritz Wagner,0.510811
462,Patrick Beverley,0.478125


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [6]:
hustle_70percent_gp.nlargest(5,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
213,Hamidou Diallo,1.364384
295,John Konchar,1.362712
487,Robert Williams III,1.350000
325,Jusuf Nurkic,1.276596
262,Jarred Vanderbilt,1.270588


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(5,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
198,Gary Payton II,5.296552
406,Matisse Thybulle,5.244094
129,De'Anthony Melton,4.571681
302,Jordan McLaughlin,4.500000
138,Dejounte Murray,4.148276


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(5,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
495,Rudy Gobert,17.318012
524,Steven Adams,16.363636
247,Jakob Poeltl,15.172318
325,Jusuf Nurkic,15.063830
215,Hassan Whiteside,14.740000


# The No Fly Zone Award (presented by Dikembe Mutumbo)*

most blocked dunks as the blocking player

In [9]:
misses=pbp2022[pbp2022.HOMEDESCRIPTION.str.startswith('MISS',na=False)| 
           pbp2022.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
missed_dunks=misses[misses.HOMEDESCRIPTION.str.contains('Dunk',na=False)|
                   misses.VISITORDESCRIPTION.str.contains('Dunk',na=False)]
blocked_dunks=missed_dunks[missed_dunks.HOMEDESCRIPTION.str.contains('BLOCK',na=False)|
                   missed_dunks.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
#as the blocker
blocked_dunks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER3_NAME,count
105,Jaren Jackson Jr.,13
10,Andre Drummond,10
44,Daniel Gafford,10
78,Giannis Antetokounmpo,10
107,Jarrett Allen,9
143,Kristaps Porzingis,9


# The Rejected for Boarding Award (sponsored by United Airlines)*

most blocked dunks as the dunking player (credit to Legdrop\_soup for the idea and asw7412 for the sponsor)

In [10]:
#as the blockee
blocked_dunks.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
201,Precious Achiuwa,11
49,Darius Bazley,10
100,Ivica Zubac,10
145,Kenyon Martin Jr.,10
211,Rudy Gobert,9


# The “Oops, I Dunked It Again” Award (sponsored by Britney Spears, presented by Gary Payton & Shawn Kemp)*

Most prolific alley-oop duo (credit to lactardenthusiast for the idea)

In [11]:
#get alley oops
alley_oops=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Alley',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Alley',na=False)]
#remove missed alley oop attempts
made_alley_oops=alley_oops[~alley_oops.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~alley_oops.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
#remove alley oops that have missing 2nd player (self-alley oop or perhaps missing data?)
made_alley_oops_complete=made_alley_oops[~made_alley_oops.PLAYER2_NAME.isna()].copy()
made_alley_oops_complete['p1']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].min(axis=1)
made_alley_oops_complete['p2']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].max(axis=1)
made_alley_oops_complete.groupby(['p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,p1,p2,count
268,Clint Capela,Trae Young,88
435,Dwight Powell,Luka Doncic,60
780,Mike Conley,Rudy Gobert,50
765,Marcus Smart,Robert Williams III,41
320,Darius Garland,Jarrett Allen,36


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [12]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)

In [13]:
hustle_50percent_gp_ranks.to_csv(path_or_buf="Output Data/Hustle Ranks.csv")

In [14]:
hustle_50percent_gp_ranks.nsmallest(n=5,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
587,Will Barton,0.152299,0.209770,0.232759,0.008621,0.106322,0.709770
533,Terrence Ross,0.152299,0.106322,0.281609,0.117816,0.074713,0.732759
289,Joe Ingles,0.152299,0.045977,0.195402,0.482759,0.025862,0.902299
22,Anfernee Simons,0.439655,0.178161,0.028736,0.011494,0.261494,0.919540
178,Eric Gordon,0.442529,0.020115,0.106322,0.287356,0.100575,0.956897


In [15]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME'].isin(['Patrick Beverley','Russell Westbrook'])].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
462,Patrick Beverley,0.988506,0.097701,0.928161,0.054598,0.896552,2.965517
497,Russell Westbrook,0.821839,0.043103,0.477011,0.295977,0.729885,2.367816


# The "Mr. Fantastic" Award (presented by Shane Battier, the No-Stats All-Star)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [16]:
hustle_50percent_gp_ranks.nlargest(n=5,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
536,Thaddeus Young,0.985632,0.813218,0.939655,0.873563,0.968391,4.580460
134,DeMarcus Cousins,0.954023,0.870690,0.945402,0.908046,0.709770,4.387931
164,Draymond Green,0.873563,0.804598,0.870690,0.928161,0.902299,4.379310
312,Josh Okogie,0.936782,0.675287,0.962644,0.669540,0.787356,4.031609
17,Alperen Sengun,0.942529,0.910920,0.873563,0.913793,0.382184,4.022989


# The Bowling Ball Award (sponsored by [Pete Weber](https://www.youtube.com/watch?v=gKQOXYB2cd8), presented by Glen "Big Baby" Davis)*

most charges committed (credit to Kdog122025 for the idea)

In [17]:
charges=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Charge',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Charge',na=False)]

charges.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
121,Giannis Antetokounmpo,23
259,Pascal Siakam,16
189,Jusuf Nurkic,14
208,Kyle Kuzma,14
22,Bam Adebayo,13


# "The Good Ol' Hockey Game, is the Best Game You Can Name" Award (presented by Dominik Hasek)*

most goaltends committed (credit to Kdog122025 for the idea)

In [18]:
goaltends=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Goaltending',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Goaltending',na=False)]
goaltends.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
39,Clint Capela,19
92,JaVale McGee,15
8,Andre Drummond,13
0,Aaron Gordon,12
158,Mo Bamba,11
162,Myles Turner,11


# The Anti-Verticality Award (presented by Roy Hibbert)*

most 3-point shooting fouls committed (credit to watchingsongsDL, kingcobweb & An-Indian-In-The-NBA for the idea)

In [19]:
third_free_throws=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)|
                          pbp2022['VISITORDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)]

shooting_fouls=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('S.FOUL',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('S.FOUL',na=False)]

fouls_on_missed_threes=shooting_fouls.merge(right=third_free_throws,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

three_pointers=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('3PT',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('3PT',na=False)]

made_threes=three_pointers[~three_pointers.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~three_pointers.VISITORDESCRIPTION.str.startswith('MISS',na=False)]

fouls_on_made_threes=shooting_fouls.merge(right=made_threes,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

fouls_on_threes=fouls_on_missed_threes.append(fouls_on_made_threes)
fouls_on_threes.groupby('PLAYER1_NAME_x').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME_x,count
93,Desmond Bane,10
23,Bam Adebayo,7
36,Buddy Hield,7
81,De'Anthony Melton,7
122,Garrett Temple,7
240,Malik Monk,7
248,Matisse Thybulle,7
255,Miles Bridges,7
350,Tyrese Maxey,7


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played (https://www.nba.com/stats/players/scoring/?sort=GP&dir=-1)

1. Luka Doncic (86%)
2. Chris Paul (84.1%)
3. Shai Gilgeous-Alexander (83.2%)
4. Trae Young (82.9%)
4. James Harden (80.5%)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

1. Davis Bertans (95.5%)
2. Wayne Ellington (95.4%)
3. Duncan Robinson & Ben McLemore (93.1%)
4. Mike Muscala & Garrison Mathews (92.9%)

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award

Lowest contested rebound percentage, minimum 50% of games played

In [20]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
232,D.J. Augustin,0.063
136,Tyus Jones,0.071
187,Trae Young,0.098
243,Isaiah Joe,0.098
271,Aaron Holiday,0.099


In [21]:
reb_w_gp_qualify[reb_w_gp_qualify['PLAYER_NAME']=='Carmelo Anthony'][['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
68,Carmelo Anthony,0.311


alternatively: restricting to players > 6 foot 6 inches in height

In [22]:
player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
46,Furkan Korkmaz,0.129
41,Duncan Robinson,0.144
4,Amir Coffey,0.152
160,Ziaire Williams,0.158
15,Cam Reddish,0.162


# The "Where There's a Will, There's a Way" Award (presented by Dennis Rodman)

Highest contested rebound percentage, minimum 50% of games played

In [23]:
reb_w_gp_qualify.nlargest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
79,Mitchell Robinson,0.605
144,Jakob Poeltl,0.579
400,Jock Landale,0.567
134,Steven Adams,0.529
60,Ivica Zubac,0.526


alternatively: restricting to players < 6 foot 7 inches in height

In [24]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
169,Thanasis Antetokounmpo,0.438
85,Ish Wainright,0.405
115,Kenrich Williams,0.391
88,Jae'Sean Tate,0.378
161,Stanley Johnson,0.376


# The "David vs Goliath" Award (presented by Dwyane Wade)*

most shots blocked with at least a 5-inch difference in height

In [26]:
blocks=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('BLOCK',na=False)|
                          pbp2022['VISITORDESCRIPTION'].str.contains('BLOCK',na=False)]

blocking_player_heights=blocks.merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER1_ID',right_on='PLAYER_ID').merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER3_ID',right_on='PLAYER_ID')
blocking_player_heights["height_difference"]=blocking_player_heights['PLAYER_HEIGHT_INCHES_x']-blocking_player_heights['PLAYER_HEIGHT_INCHES_y']
blocking_player_heights.query('height_difference >= 5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER3_NAME,count
194,Patrick Beverley,27
87,Fred VanVleet,25
84,Facundo Campazzo,18
231,Terry Rozier,16
67,Derrick White,15


# The Rotation Awards

**value depth over one solitary star. used basketball-reference's play-by-play position percentages as baseline for position groupings, minimum 250 MP & 12 MP/G**

# The Best Guard Rotation Award (sponsored by Buckingham Palace)

East:
1. Heat (Lowry, Herro, Vincent)
2. Bucks (Jrue, George Hill, DiVincenzo, Jevon Carter)
3. Sixers (*Harden*, Maxey, Seth, Shake)
4. Bulls (Lonzo, Caruso, Coby, Ayo)
5. Boston (Smart, Schroder, Fast PP, Maximum Derrick)
6. Cavaliers (Garland, Rubio, Sexton, LeVert, Rondo, Brandon Goodwin)
7. Raptors (FVV, GTJ)
8. Nets (Harden, *Seth*, Patty Mills, [World B Flat](https://www.basketball-reference.com/players/i/irvinky01.html), Jevon Carter, Dragic, David Duke Jr.)
9. Hornets (LaMelo, Scary Terry, Ish Smith)
10. Hawks (Trae, Delon, Lou Will)
11. Wizards (Beal, Dinwiddie, Neto, Aaron Holiday, Ish Smith)
12. Knicks (Quickley, Kemba, Grimes, Rose, Burks)
13. Pacers (Duarte, Brogdon, Haliburton, Duane Washington, TJ McConnell, Keifer Sykes, Brad Wanamaker)
14. Pistons (Cory Jo, Killian, Frank Jackson, Saben Lee)
15. Magic (Cole Anthony, Suggs, Hampton, Fultz)

West:

1. Suns (CP3, Cam Payne, Aaron Holiday)
2. Grizzlies (Ja, Melton, Tyus Jones, Konchar)
3. Warriors (Steph, Poole, GPII)
4. Jazz (Spida, Conley, Trent Forrest, *Clarkson*)
5. Mavericks (Brunson, Luka, Frankie Smokes, Dinwiddie)
6. Nuggets (Monte Morris, Campazzo, Bones)
7. Wolves (D'Angelo Russell, Malik Beasley, Pat Bev, Jordan McLaughlin)
8. Clippers (Big Government, Kennard, Bledsoe)
9. Lakers (Westbrook, Avery Bradley, Rondo, DJ Augustin)
10. Pelicans (Devonte' Graham, McCollum, Garrett Temple, Jose Alvarado, Kira Lewis)
11. Spurs (Dejounte, Maximum Derrick, Tre Jones, Bryn Forbes)
12. Trail Blazers (Simons, Dame, DSJ, *McCollum*, *Norm Powell*, Brandon Williams, Kris Dunn)
13. Kings (Fox, Hali, Davion Mitchell)
14. Thunder (SGA, Tre Mann, Ty Jerome, Theo Maledon)
15. Rockets (Jalen Green, KPJ, Armoni Brooks, DJ Augustin, Schroder)

# The Best Wing Rotation Award (co-sponsored by Lou Williams and Magic City)

East:
1. Heat (Butler, Strus, [Sheen](https://twitter.com/thenextonepod/status/1307500403924369409), Caleb Martin, Oladipo)
2. Bucks (Middleton, Connaughton, Jordan Nwora, Hood, Wesley Matthews, [Ted Cruz](https://www.cbssports.com/college-basketball/news/look-ted-cruz-hilariously-embraces-grayson-allen-comparison-on-twitter/))
3. Sixers (Niang, Danny Green, Korkmaz, Thybulle)
4. Bulls (LaVine, Deebo, Javonte Green, TBJ, DJJ)
5. Boston (Jay^2, Josh Richardson, Langford)
6. Cavaliers (Okoro, Cedi, Lamar Stevens, Dean Wade)
7. Raptors (Scottie, Anunoby, Svi)
8. Nets (Bembry, James Johnson, Joe Harris, Bruce Brown, Cam Thomas, Kessler Edwards)
9. Hornets (Miles Bridges, Hayward, Jalen McDaniels, Oubre, Cody Martin)
10. Hawks (Huerter, Bogdan^2, Hunter, Reddish, TLC)
11. Wizards (KCP, Corey Kispert, Deni Avdija, Hachimura, Satoransky)
12. Knicks (RJ, Fournier)
13. Pacers (Justin Holiday, Oshae, LeVert, Lamb, Lance, Buddy Hield, Terry Taylor, Kelan Martin)
14. Pistons (Saddiq Bey, Josh Jackson, Cade, Hamidou, McGruder)
15. Magic (Franz, Okeke, Gary Harris, Terrence Ross)

West:

1. Suns (Book, Bridges, Waluigi, Crowder, Torrey Craig)
2. Grizzlies (Bane, Ziaire Williams, Dillon Brooks, *Kyle Anderson*)
3. Warriors (Wiggins, JTA, Iggy, Klay, Damion Lee)
4. Jazz (Royce O'Neale, Bojan, Ingles, Danuel House)
5. Mavericks (DFS, Bullock, THJ, Josh Green, Sterling Brown)
6. Nuggets (Aaron Gordon, JaMychal, MPJ, Will Barton, Austin Rivers, Davon Reed, Bryn Forbes, PJ Dozier)
7. Wolves (Jaden McDaniels, Taurean Prince, Anthony Edwards, Jaylen Nowell)
8. Clippers (Terance Mann, Marcus Morris, PG13, Covington, Amir Coffey, Brandon Boston)
9. Lakers (LeBron, Ariza, Monk, THT, Reaves, Ellington, Bazemore)
10. Pelicans (Josh Hart, NAW, Herb Jones, Brandon Ingram, Trey Murphy, Naji Marshall, Tomas Satoransky, Tony Snell)
11. Spurs (Big Body, Keita Bates-Diop, Vassell, Lonnie Walker, Primo, Josh Richardson)
12. Trail Blazers (Nassir Little, CJ Elleby, McLemore, Tony Snell, Josh Hart, Justise Winslow, Keon Johnson)
13. Kings (Hield, Terence Davis, Justin Holiday, DiVincenzo, Harrison Barnes, Harkless)
14. Thunder (Giddey, Aaron Wiggins, DORT, Bazley, Kenrich, Lindy Waters, Vit Krejci)
15. Rockets (Jae'Sean Tate, Kenyon Martin Jr, Nwaba, Eric Gordon, Garry Bird, Josh Christopher)

# The Best Big Rotation Award (jointly sponsored by Tom Hanks, Cadbury and Sex and the City)

East:

1. Heat (Bam, Yurt7, Dedmon, PJ Tucker, Markieff)
2. Bucks (Giannis, Bobby Portis, Semi Ojeleye, Ibaka, Cousins, Brook Lopez)
3. Sixers (Embiid, Tobi, Drummond)
4. Bulls (Vooch, Tristan Thompson, Patrick Williams)
5. Boston (Horford, Timelord, Theis, *Grant Williams*)
6. Cavaliers (Allen, Mobley, Markkanen, *Love*)
7. Raptors (Siakam, ~~Pascal~~, Achiuwa, Khem Birch, *Boucher*, *Thad Young*)
8. Nets (KD, LMA, Blake Griffin, Claxton, Drummond, Day'Ron Sharpe)
9. Hornets (Mason Plumlee, Montrezl Harrell, PJ Washington)
10. Hawks (Capela, John Collins, Gallinari, Okongwu)
11. Wizards (Kuzma, Montrezl, Gafford, Thomas Bryant, Porzingis, Bertans)
12. Knicks (Randle, Mitch Rob, Obi, Taj Gibson, Nerlens)
13. Pacers (Sabonis, Myles Turner, Torrey Craig, Goga, IJax, Jalen Smith)
14. Magic (WCJ, Bamba, Mo Wagner, Robin Lopez)
15. Pistons (Beef Stew, Jerami Grant, Lyles, Olynyk, Bagley)

West:

1. Suns (Cam Johnson, Ayton, JaVale, Jalen Smith, Bismack, Kaminsky)
2. Grizzlies (JJJ, Steven Adams, Killian Tillie, *Brandon Clarke*, *Xavier Tillman*)
3. Warriors (Looney, OPJ, Dray, Bjelica, Kuminga)
4. Jazz (Gobert, Whiteside, Rudy Gay, Paschall)
5. Mavericks (Dwight Powell, Kleber, Porzingis, Bertans)
6. Nuggets (Jokic, Jeff Green, Nnaji, DeMarcus Cousins)
7. Wolves (KAT, Vanderbilt, Naz Reid)
8. Clippers (Zubac, Batum, Hartenstein, Ibaka, Winslow)
9. Lakers (Carmelo, Day-to-Dayvis, Dwight, Stanley Johnson, DeAndre Jordan)
10. Pelicans (Valanciunas, Jaxson Hayes, Willy Hernangomez)
11. Spurs (Poeltl, Doug McDermott, Eubanks, Thad Young, Zach Collins)
12. Trail Blazers (Nurk, Covington, Larry Nance Jr, Trendon Watford, Eubanks, Cody Zeller)
13. Kings (Richaun, Sabonis, Metu, Bagley, Len, Damian Jones, Tristan Thompson, *Trey Lyles*)
14. Thunder (JRE, Poku, Favors, Muscala, Roby, Olivier Sarr)
15. Rockets (Christian Wood, Alperen, Theis)